# カチャカ_ライントレース
2023国際ロボット展で展示したX7とカチャカの搬送デモで使用したコードです。  
カチャカの前方カメラ画像を用いてライントレースをします。  
詳細はアールティ移動型ロボットブログの「ROS 2でカチャカを動かしてみた」の連載を御覧ください。  
[https://rt-net.jp/mobility/archives/24768](https://rt-net.jp/mobility/archives/24768)

In [ ]:
import cv2
from cv2 import aruco
import kachaka_api
import numpy as np
import time
from IPython.display import Image, clear_output, display

In [ ]:
client_info = kachaka_api.KachakaApiClient()


dictionary = aruco.getPredefinedDictionary(aruco.DICT_6X6_50)
parameters = aruco.DetectorParameters()

camera_info = client_info.get_front_camera_ros_camera_info()

mtx = np.array(camera_info.K, dtype=float).reshape(3, 3)
dist = np.array(camera_info.D)

In [ ]:
def detect_line(cv_image):
    hsv = cv2.cvtColor(cv_image, cv2.COLOR_BGR2HSV)

    lower_yellow = np.array([20, 80, 10])  
    upper_yellow = np.array([40, 255, 255])  

    lower_white = np.array([0, 0, 200], dtype=np.uint8)  
    upper_white = np.array([180, 25, 255], dtype=np.uint8)

    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([130, 255, 255])

    lower_pink = np.array([140, 50, 50]) 
    upper_pink = np.array([170, 255, 255])  

    lower_green = np.array([30, 64, 40])  
    upper_green = np.array([90, 255, 255])
    
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    
    res = cv2.bitwise_and(cv_image, cv_image, mask=mask)
    h, w = cv_image.shape[:2]
    
    range = (h//4)*3
    mask[0:range, 0:w] = 0
    mask[range+30:h, 0:w] = 0
    
    return mask, res

In [ ]:
client = kachaka_api.aio.KachakaApiClient()
STARTFLAG = False
marker_length = 0.04
stop_distance = 0.3
wait_timer = 10

以下実行後、スマートフォンなどで https://pf-robotics.github.io/textcode/ にアクセスし、QRコードをカチャカの前カメラの前に近付けてコマンドを指令して下さい。  
コマンド一覧
- home：充電ドックに戻る
- dock：シェルフにドッキング
- unlock：シェルフを解除
- start：ライントレース開始
- stop：ライントレース停止

In [ ]:
qcd = cv2.QRCodeDetector()
await client.speak("起動しました")
async for image in client.front_camera_ros_compressed_image.stream():
    cv_image = cv2.imdecode(np.frombuffer(image.data, dtype=np.uint8), flags=1)
    decoded_info, corners, _ = qcd.detectAndDecode(cv_image)
    
    if decoded_info != "":
        cv_image = cv2.putText(
            cv_image,
            decoded_info,
            corners[0][0].astype(int),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            cv2.LINE_AA,
        )

        if decoded_info == "home":
            line_tracer == False
            await client.return_home(cancel_all=True,tts_on_success="ホームに戻りました")
        if decoded_info == "dock":
            await client.dock_shelf(cancel_all=True)
        if decoded_info == "unlock":
            await client.undock_shelf(cancel_all=True)
        if decoded_info == "start":
            STARTFLAG = True
            await client.speak("ライントレースを開始します")
        if decoded_info == "stop":
            STARTFLAG = False
            await client.speak("待機します")

    if STARTFLAG == True:
        h, w = cv_image.shape[:2]
        corners_ar, ids, rejectedImgPoints = aruco.detectMarkers(cv_image, dictionary)
    
        if corners_ar:
            for i in range(len(ids)):
                rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(corners_ar, marker_length, mtx, dist)
                x = tvec[0][0][0]
                z = tvec[0][0][2]
                if  z  < stop_distance:
                    time.sleep(wait_timer)
                    pose = await client.get_robot_pose()
                    await client.move_to_pose(pose.x, pose.y, pose.theta - np.pi)
                    continue
    
        mask, res = detect_line(cv_image)
        
        M = cv2.moments(mask)
        if M['m00'] > 0 :	
            cx,cy = int(M['m10']/M['m00']), int(M['m01']/M['m00'])
            	
            cv2.circle(cv_image, (cx, cy), 20, (0, 0, 255), -1)
            diff = cx - w//2	
            angle = -float(diff)/1500
            await client.set_robot_velocity(linear=0.10, angular=angle)
            
    _, ret = cv2.imencode(
        ".jpg",
        cv2.resize(cv_image, (int(cv_image.shape[1] / 2), int(cv_image.shape[0] / 2))),
    )
    clear_output(wait=True)
    display(Image(data=ret, format="jpeg"))